In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

In [2]:
df = pd.read_csv("Master Hockey Data.csv")

In [3]:
to_drop = ['TOI/GP','S%','FOW%', "S"]
df.drop(to_drop, inplace=True, axis=1)

In [4]:
# df.head()
# need to remove index 2484 because it shows as a float and fuckes everything up
indexRemove = df[df['S/C'] == "--" ].index
df.drop(indexRemove , inplace=True)
df.drop(df.index[2484], inplace = True)

In [5]:
encoder = preprocessing.LabelEncoder()
data = df.values
values = encoder.fit_transform(data[:,1])
data[:,1] = values

In [6]:
values = encoder.fit_transform(data[:, 2])

In [7]:
data[:,2] = values

In [8]:
def myFunc(str):
    if str == "--" or type(str) == 'float':
        return "0"
    return str.replace(",","")

vFunc = np.vectorize(myFunc)
cols = [3,5,6,8,10,11,12,13,14,15]
for col in cols:
    data[:, col] = vFunc(data[:,col]).astype(float)
data[0,:]

array(['Wayne Gretzky', 0, 0, 1487.0, 894.0, 1963.0, 2857.0, 520.0, 577.0,
       1.92, 617.0, 1818.0, 204.0, 890.0, 73.0, 149.0, 2.0, 91.0],
      dtype=object)

In [9]:
my_data = pd.read_csv("Master HOF Data.csv")
hof_names = my_data["Player"].tolist()

    

In [10]:
labels = []
for i in range(np.shape(data)[0]):
    if data[i,0] in hof_names:
        labels.append(1)
    else:
        labels.append(0)

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(data[:, 1:], labels , test_size=0.30)

In [12]:
model = tree.DecisionTreeClassifier()
model.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [13]:
pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, pred))

Accuracy: 0.9502103786816269
